In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [11]:
#Reading Data
df = pd.read_csv("dataset.csv")
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
test_df, val_df = train_test_split(test_df, test_size=0.5, random_state=42)
X_train = train_df.drop(columns=['Unnamed: 0', 'smoking']).to_numpy()
y_train = train_df[['smoking']].to_numpy().reshape(-1) #Reshape from (n,1) to (n)

X_val = val_df.drop(columns=['Unnamed: 0', 'smoking']).to_numpy()
y_val = val_df[['smoking']].to_numpy().reshape(-1)

num_samples, num_features = X_train.shape

In [18]:
from sklearn.tree import DecisionTreeClassifier

class MyRandomForest():
    def __init__(self, num_trees, max_height, max_features):
        self.num_trees = num_trees
        self.max_height = max_height
        self.max_features = max_features
        self.trees = []

    def train(self, X_train, y_train):
        num_samples  = X_train.shape[0]        
        for i in range(self.num_trees):
            samples = np.random.choice(num_samples, num_samples)
            sampled_X = X_train[samples]
            sampled_Y = y_train[samples]
            tree = DecisionTreeClassifier(max_depth=self.max_height, max_features=self.max_features)
            tree.fit(sampled_X, sampled_Y)
            self.trees.append(tree)


    def predict(self, X):
        sum_predictions = np.zeros(X.shape[0])
        for tree in self.trees:
            prediction = tree.predict(X)
            sum_predictions +=  np.where(prediction == 0, -1, prediction)  #np.where used to replace 0s with -1s         
        return np.where(sum_predictions > 0, 1, 0)
    

    def score(self, X, y):
        prediction = self.predict(X)
        return (prediction == y).sum()/  X.shape[0]   
    








In [26]:
forest = MyRandomForest(num_trees=50, max_height=12, max_features=3)

forest.train(X_train, y_train)

print(forest.score(X_train, y_train))
print(forest.score(X_val, y_val))

0.7778562682490345
0.7522918498053497
